In [3]:
import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from scipy.stats import circmean, circstd

import healpy as hp
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord as SkyCoord

In [4]:
nside = 64
npix = hp.nside2npix(nside)

In [5]:
# Elimando fonte que estejam com B em [-b,b]
def galactic_cut(data,RA_column_index,Dec_column_index,Lim_inf,Lim_sup):

    data_array = np.array(data)

    ra_column_array = data_array[:,RA_column_index]
    dec_column_array = data_array[:,Dec_column_index]

    c = SkyCoord(ra_column_array,dec_column_array, frame='icrs', unit='deg')

    b_array = np.array(c.galactic.b)

    Index = 0
    Index_list = []
    for b in b_array:
        if b > Lim_inf  and b < Lim_sup:
            Index_list.append(Index)
        Index+=1

    index_to_delete = np.array(Index_list)
    new_data_array = np.delete(data_array,index_to_delete,axis=0)
    new_data = pd.DataFrame(new_data_array,columns=data.columns.values)

    return new_data

In [6]:
name = 'NVSS'
filename = 'NVSS_Catalog.fits'
hdu = fits.open(filename)
nvss = hdu[1].data
nvss = Table(nvss)
nvss = nvss.to_pandas()

# Corte de Fluxo 20 mJy < S < 1000 mJy
nvss = nvss[(20 <= nvss['FLUX_20_CM']) & (nvss['FLUX_20_CM'] <= 1000)]

cut_b = 20
nvss = galactic_cut(nvss,0,1,-cut_b,cut_b)

C_nvss = SkyCoord(nvss['RA'],nvss['DEC'], frame='icrs', unit='deg')
l, b  = C_nvss.galactic.l.value, C_nvss.galactic.b.value

In [7]:
data = nvss 
nside = 64
npix = hp.nside2npix(nside)
# Obtendo os angulos dos pixels em coordenadas esféricas padrão em radianos
thetas_pixels, phis_pixels = hp.pix2ang(nside, np.arange(npix))
# obtendo as coordenadas centrais cartesianas de cada pixel
central_pixels_positions = hp.ang2vec(thetas_pixels,phis_pixels)

# obtendo as coordenadas cartesianas de cada fontes
#l, b = data['l'], data['b']
phis_fontes, thetas_fontes = np.radians(l), np.radians(90.0 - b)
coord_ca_cartesian_array = hp.ang2vec(thetas_fontes,phis_fontes)

pix_indices = hp.ang2pix(nside, thetas_fontes, phis_fontes)
# Contando o número de fontes em cada pixel
density_map = np.bincount(pix_indices, minlength=npix)

In [8]:
def exclui_pixels_vazios(density_map,thetas_pixels,phis_pixels):
    
    indices_para_excluir = np.where(density_map==0)[0]
    density_map = np.delete(density_map,indices_para_excluir)

    thetas_pixels = np.delete(thetas_pixels,indices_para_excluir)
    phis_pixels = np.delete(phis_pixels,indices_para_excluir)

    n_x = np.sin(thetas_pixels)*np.cos(phis_pixels)
    n_y = np.sin(thetas_pixels)*np.sin(phis_pixels)
    n_z = np.cos(thetas_pixels)

    n_vec = np.stack([n_x,n_y,n_z],axis=1)

    return density_map, n_vec

In [9]:
Ni, n_vec = exclui_pixels_vazios(density_map,thetas_pixels,phis_pixels)

In [10]:
M, d_vec = np.mean(density_map), np.array([0.01, 0.01, 0.01])
initial_guess = [M, d_vec[0], d_vec[1], d_vec[2]]

In [ ]:
def chi2(params):
    M0 = params[0]
    d_vec = params[1:]
    
    N_i_model = N_i_model = M0 + np.dot(n_vec,d_vec)
    
    chi2_value = np.sum((Ni - N_i_model)**2 / Ni)
    
    return chi2_value

print('') 
print('Minimizando χ2')
initial_guess = [M, d_vec[0], d_vec[1], d_vec[2]]

print('Rodando uma única vez (Método Nelder-Mead)')
result = minimize(chi2, initial_guess, method='Nelder-Mead')

M0_best, Dx_best, Dy_best, Dz_best = result.x[0], result.x[1], result.x[2], result.x[3]

# anisotropia_dipolar:
D_modulo_relativo = np.sqrt((Dx_best**2)+(Dy_best**2)+(Dz_best**2))/M0_best
print(f'|D_relativo|={D_modulo_relativo:.4f}')

D_modulo_absoluto = np.sqrt(Dx_best**2 + Dy_best**2 + Dz_best**2)
print(f'|D_absoluto|={D_modulo_absoluto:.3f}')

n = np.array([Dx_best,Dy_best,Dz_best])/np.linalg.norm(D_modulo_absoluto)

theta_n, phi_n = hp.vec2ang(n)
l_result, b_result  = np.degrees(phi_n), 90 - np.degrees(theta_n)
l_result, b_result


Minimizando χ2
Rodando uma única vez (Método Nelder-Mead)
|D_relativo|=0.0294
|D_absoluto|=0.189


(array([232.39590568]), array([66.62910383]))

Craindo mapas simulados

Usando **np.random.poisson(Np)** em capa pixel, onde **Np** é a contidade de fontes do catálogo original

In [20]:
def mapa_simulado_mask(density_map):

    map = []
    for Np in density_map:

        N_simulado = np.random.poisson(Np)
        map.append(N_simulado)
    # Convertendo density_map para float64 para suportar hp.UNSEEN
    map = np.array(map)
    novo_map = map.astype(np.float64)
    # Criando a máscara para os pixels onde não há fontes (valor zero)
    mask = novo_map > 0
    # Aplicando a máscara ao mapa para criar um mapa mascarado
    masked_map = np.copy(novo_map)
    masked_map[~mask] = hp.UNSEEN
    return masked_map, np.sum(map)

def mapa_simulado_unmask(density_map):

    map = []
    for Np in density_map:

        N_simulado = np.random.poisson(Np)
        map.append(N_simulado)
    # Convertendo density_map para float64 para suportar hp.UNSEEN
    map = np.array(map)
    novo_map = map.astype(np.float64)
    return novo_map, np.sum(novo_map)

In [40]:
def erros(num, density_map):
    #num = 1000

    D_re_lista = []
    D_abs_lista = []
    l_lista=[]
    b_lista=[]
    for _ in range(num):

        simulado, total_fontes = mapa_simulado_unmask(density_map)
        Ni1, n_vec1 = exclui_pixels_vazios(simulado,thetas_pixels,phis_pixels)

        #print('Quantidade de fontes:',total_fontes)

        def chi2(params):
            M0 = params[0]
            d_vec = params[1:]
            
            N_i_model = N_i_model = M0 + np.dot(n_vec1,d_vec)
            
            chi2_value = np.sum((Ni1 - N_i_model)**2 / Ni1)
            
            return chi2_value
        
        #print('Novo array density_map:',Ni1)
        #print('Quantidade de pixels visiveis:',len(Ni1))

        M, d_vec1 = np.mean(simulado), np.array([0.01, 0.01, 0.01])

        #print('') 
        #print('Minimizando χ2')
        initial_guess = [M, d_vec1[0], d_vec1[1], d_vec1[2]]

        #print('Rodando uma única vez (Método Nelder-Mead)')
        result1 = minimize(chi2, initial_guess, method='Nelder-Mead')

        M0_best1, Dx_best1, Dy_best1, Dz_best1 = result1.x[0], result1.x[1], result1.x[2], result1.x[3]

        # anisotropia_dipolar:
        D_modulo_relativo1 = np.sqrt((Dx_best1**2)+(Dy_best1**2)+(Dz_best1**2))/M0_best1
        #print(f'|D_relativo|={D_modulo_relativo1:.4f}')
        D_re_lista.append(D_modulo_relativo1)

        D_modulo_absoluto1 = np.sqrt(Dx_best1**2 + Dy_best1**2 + Dz_best1**2)
        #print(f'|D_absoluto|={D_modulo_absoluto1:.3f}')
        D_abs_lista.append(D_modulo_absoluto1)

        n1 = np.array([Dx_best1,Dy_best1,Dz_best1])/np.linalg.norm(D_modulo_absoluto1)

        theta_n1, phi_n1 = hp.vec2ang(n1)
        l_result1, b_result1  = np.degrees(phi_n1), 90 - np.degrees(theta_n1)

        #print(l_result1, b_result1)
        l_lista.append(l_result1)
        b_lista.append(b_result1)

    D_re_array = np.array(D_re_lista)
    D_re_mean, D_re_std = np.mean(D_re_array), np.std(D_re_array)
    print(f'D_rel = {D_re_mean:.3f} ± {D_re_std:.3f}')

    D_abs_array = np.array(D_abs_lista)
    D_abs_mean, D_abs_std = np.mean(D_abs_array), np.std(D_abs_array)
    
    print(f'D_abs = {D_abs_mean:.2f} ± {D_abs_std:.2f}')

    # Latitude Galáctica
    l_lista = np.array(l_lista)
    l_rad = np.radians(l_lista)
    # Latitude Média
    l_mean_rad = circmean(l_rad, high=2*np.pi, low=0)
    l_mean_deg = np.degrees(l_mean_rad)
    # Latitude desvio 
    l_std_rad = circstd(l_rad, high=np.pi, low=-np.pi)
    l_std_deg = np.degrees(l_std_rad)
    #print(f'l: {l_mean_deg:.2f}°')
    #print(f'dl: {l_std_deg:.2f}°')

    # Longitude Galáctica
    b_lista = np.array(b_lista)
    b_rad = np.radians(b_lista)
    # Longitude Média
    b_mean_rad = circmean(b_rad, high=np.pi, low=-np.pi)
    b_mean_deg = np.degrees(b_mean_rad)
    # Longitude desvio
    b_std_rad = circstd(b_rad, high=np.pi, low=-np.pi)
    b_std_deg = np.degrees(b_std_rad)
    #print(f'b: {b_mean_deg:.2f}°')
    #print(f'db:{b_std_deg:.2f}°')

    print(f'(l,b) = {l_mean_deg:.2f}°±{l_std_deg:.2f}°, {b_mean_deg:.2f}°±{b_std_deg:.2f}°')
    return 0

In [41]:
erros(1000,density_map)

D_rel = 0.030 ± 0.006
D_abs = 0.16 ± 0.04
(l,b) = 223.11°±35.74°, 54.89°±13.35°


0

Erros ainda são muito grandes